<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install transformers datasets captum quantus accelerate bitsandbytes

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from datetime import datetime
from datasets import Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from captum.attr import IntegratedGradients, InputXGradient

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# ===================================================
# 1. KONFIGURACJA GLOBALNA
# ===================================================

# === Parametry analizy XAI ===
N_SAMPLES_XAI = (
    100  # Liczba próbek dla metod XAI (Integrated Gradients / InputXGradient)
)
XAI_N_STEPS = 50  # Zwiększono z 10 na 50 dla lepszej stabilności przybliżenia całki
N_SAMPLES_PROBE = 1000  # Liczba próbek do analizy warstwowej metodą RepE
N_SAMPLES_STABILITY = 50  # Liczba par tekst-parafraza do testu stabilności
BATCH_SIZE = (
    32  # Rozmiar batcha dla przetwarzania wsadowego (optymalizacja pamięci GPU)
)
TOP_K_TOKENS = (
    5  # Liczba najważniejszych tokenów do usunięcia w metryce Comprehensiveness
)
DF_SIZE = 3000  # Ograniczenie wielkości zbioru danych (dla szybszego testowania)

# === Długość sekwencji ===
MAX_SEQUENCE_LENGTH = 256  # Maksymalna długość sekwencji tokenów

# === Indeksy warstw do analizy ===
TARGET_LAYER_INDEX = 5  # Warstwa docelowa do analizy (warstwa 5 wykazała najlepszą separowalność liniową)
STEERING_ALPHA = -3.0  # Siła wektora sterującego (ujemna wartość = detoksykacja)

# === Próg klasyfikacji ===
CLASSIFICATION_THRESHOLD = 0.5  # Próg prawdopodobieństwa dla klasyfikacji binarnej

# === Parametry testu stabilności (Moduł C) ===
PARAPHRASE_MIN_SIMILARITY = 0.7  # Minimalny cosine similarity dla akceptacji parafrazy
PARAPHRASE_SEED = 42  # Seed dla reproducibility generowania parafraz

# === Ścieżki ===
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"

# Dodaj timestamp do nazwy katalogu, aby nie nadpisywać poprzednich wyników
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_DIR = f"/drive/MyDrive/msc-project/results_final_{TIMESTAMP}"

MODEL_CHECKPOINT = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full_20260125_133112"
# MODEL_CHECKPOINT = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full"

# === Urządzenie obliczeniowe ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Uruchomiono na urządzeniu: {device}")

# Tworzenie katalogu wyników
os.makedirs(RESULTS_DIR, exist_ok=True)


In [ ]:
# ===================================================
# 2. PRZYGOTOWANIE DANYCH I MODELU
# ===================================================


def clean_text(example):
    """
    Czyści tekst komentarza, usuwając niepożądane elementy i normalizując format.

    Funkcja stosowana jest zarówno podczas treningu jak i ewaluacji, aby zapewnić
    spójność przetwarzania danych.

    Argumenty:
        example: Słownik zawierający klucz 'comment_text' z tekstem do oczyszczenia

    Zwraca:
        Zmodyfikowany słownik example z oczyszczonym tekstem w polu 'comment_text'

    Operacje czyszczenia:
        - Konwersja na małe litery (wymagane dla modeli BERT typu uncased)
        - Usunięcie linków URL (http/https/www)
        - Usunięcie adresów IP
        - Usunięcie metadanych Wikipedii (talk pages, timestampy UTC)
        - Normalizacja białych znaków (spacje, newline, non-breaking space)
        - Usunięcie cudzysłowów z początku i końca
    """
    text = example["comment_text"]
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = re.sub(r"\(talk\)", "", text)
    text = re.sub(r"\d{2}:\d{2}, \w+ \d{1,2}, \d{4} \(utc\)", "", text)
    text = text.replace("\n", " ").replace("\xa0", " ")
    text = text.strip(' "')
    text = re.sub(r"\s+", " ", text).strip()
    example["comment_text"] = text
    return example


def prepare_environment():
    """
    Przygotowuje środowisko eksperymentalne: wczytuje dane, tokenizuje i ładuje model.

    Zwraca:
        Tuple zawierający:
        - model: Wytrenowany model DistilBERT do klasyfikacji toksyczności
        - tokenizer: Tokenizer dopasowany do modelu
        - eval_dataset: Zbiór testowy z przetworzonymi danymi

    Kroki przygotowania:
        1. Wczytanie danych z pliku CSV
        2. Preprocessing tekstów
        3. Ładowanie tokenizera
        4. Tokenizacja tekstów (padding do MAX_SEQUENCE_LENGTH)
        5. Przygotowanie etykiet binary classification
        6. Podział na zbiór treningowy i testowy
        7. Załadowanie wytrenowanego modelu
    """
    print(">>> [SETUP] Wczytywanie i przetwarzanie danych...")

    # 1. Wczytanie danych
    try:
        df = pd.read_csv(DATA_PATH).head(DF_SIZE)
        dataset = Dataset.from_pandas(df)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"Nie znaleziono pliku: {DATA_PATH}. Sprawdź ścieżkę w Konfiguracji Globalnej."
        )

    # 2. Preprocessing
    dataset = dataset.map(clean_text)

    # 3. Ładowanie tokenizera zgodnego z modelem
    print(f">>> [SETUP] Ładowanie tokenizera z: {MODEL_CHECKPOINT}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    except OSError:
        print(
            f"Błąd: Nie znaleziono tokenizera w {MODEL_CHECKPOINT}. Pobieram domyślny 'distilbert-base-uncased'."
        )
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    # 4. Tokenizacja
    def tokenize_function(examples):
        """Tokenizuje teksty z paddingiem do stałej długości MAX_SEQUENCE_LENGTH."""
        return tokenizer(
            examples["comment_text"],
            padding="max_length",
            truncation=True,
            max_length=MAX_SEQUENCE_LENGTH,
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # 5. Przygotowanie etykiet binary classification
    label_cols = [
        "toxic",
    ]

    def create_labels(example):
        """Pobiera kolumnę 'toxic' i tworzy etykietę."""
        example["labels"] = [float(example[col]) for col in label_cols]
        return example

    final_dataset = tokenized_dataset.map(create_labels)

    # Ustawienie formatu PyTorch (usunięcie kolumn tekstowych, zachowanie tylko tensorów)
    cols_to_keep = ["input_ids", "attention_mask", "labels"]
    final_dataset.set_format("torch", columns=cols_to_keep)

    # 6. Podział na zbiór treningowy i testowy
    splits = final_dataset.train_test_split(test_size=0.2, seed=42)
    eval_dataset = splits["test"]

    # 7. Ładowanie wytrenowanego modelu
    print(f">>> [SETUP] Ładowanie wytrenowanego modelu z: {MODEL_CHECKPOINT}...")
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_CHECKPOINT, num_labels=1, problem_type="single_label_classification"
        )
    except OSError:
        raise OSError(
            f"Nie znaleziono modelu w ścieżce: {MODEL_CHECKPOINT}. Upewnij się, że najpierw uruchomiłeś skrypt treningowy."
        )

    # Przełączenie w tryb ewaluacji (wyłącza dropout i batch normalization)
    model.to(device)
    model.eval()

    print(f">>> [SETUP] Środowisko gotowe. Urządzenie: {device}")
    return model, tokenizer, eval_dataset


# Inicjalizacja środowiska
model, tokenizer, eval_dataset = prepare_environment()


In [ ]:
# ===================================================
# 3. MODUŁ A: PORÓWNANIE METOD XAI (Comprehensiveness)
# ===================================================


def run_module_a_xai(model, tokenizer, dataset):
    """
    Porównuje metody XAI (Integrated Gradients vs InputXGradient) pod kątem wierności wyjaśnień.

    Metryka Comprehensiveness mierzy, jak bardzo usuniecie najważniejszych tokenów
    (zidentyfikowanych przez metodę XAI) wpływa na pewność predykcji modelu.
    Wyższy spadek pewności = lepsza metoda XAI.

    Argumenty:
        model: Wytrenowany model klasyfikacyjny DistilBERT
        tokenizer: Tokenizer odpowiadający modelowi
        dataset: Zbiór danych z etykietami

    Zwraca:
        DataFrame z wynikami porównania metod (drop scores dla IG i IxG)

    Metodologia:
        1. Wybór podzbioru toksycznych przykładów (N_SAMPLES_XAI)
        2. Dla każdego przykładu:
            a) Obliczenie oryginalnego prawdopodobieństwa toksyczności
            b) Identyfikacja TOP_K_TOKENS najważniejszych tokenów (IG i InputXGradient)
            c) Maskowanie tych tokenów i ponowna predykcja
            d) Obliczenie spadku pewności (comprehensiveness score)
        3. Wizualizacja wyników jako boxplot
    """
    print("\n>>> [MODUŁ A] Uruchamianie porównania metod XAI (IG vs IxG)...")
    model.eval()

    # Filtrowanie tylko toksycznych przykładów (indeks 0 = etykieta 'toxic')
    toxic_indices = [i for i, labels in enumerate(dataset["labels"]) if labels[0] == 1]
    subset_indices = toxic_indices[:N_SAMPLES_XAI]
    subset = dataset.select(subset_indices)

    results = []

    # Funkcja pomocnicza dla Captum (zwraca logity na podstawie embeddings)
    def predict_func(inputs_embeds, attention_mask=None):
        """Wrapper predykcji dla biblioteki Captum."""
        return model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits

    ig = IntegratedGradients(predict_func)
    ixg = InputXGradient(predict_func)

    for i in tqdm(range(len(subset)), desc="Ewaluacja XAI"):
        input_ids = subset[i]["input_ids"].unsqueeze(0).to(device)
        attention_mask = subset[i]["attention_mask"].unsqueeze(0).to(device)
        input_embeds = model.distilbert.embeddings(input_ids)

        # Baseline = embedding tokena [PAD] (punkt odniesienia dla IG)
        baseline = model.distilbert.embeddings(
            torch.tensor(
                [tokenizer.pad_token_id] * MAX_SEQUENCE_LENGTH, device=device
            ).unsqueeze(0)
        )

        # 1. Oryginalne prawdopodobieństwo toksyczności
        with torch.no_grad():
            orig_out = model(inputs_embeds=input_embeds, attention_mask=attention_mask)
            orig_prob = torch.sigmoid(orig_out.logits)[0, 0].item()

        # Funkcja pomocnicza do obliczania spadku pewności
        def calculate_drop(attr_tensor):
            """
            Oblicza spadek pewności po usunięciu TOP_K najważniejszych tokenów.

            Argumenty:
                attr_tensor: Tensor atrybutów z metody XAI

            Zwraca:
                Spadek prawdopodobieństwa (orig_prob - new_prob)
            """
            # Suma po wymiarze embeddingów -> ważność na poziomie tokenów
            attr_sum = attr_tensor.sum(dim=-1).squeeze(0)
            # Znajdź TOP_K najważniejszych tokenów
            _, top_indices = torch.topk(attr_sum, k=TOP_K_TOKENS)

            # Maskowanie tokenów (zamiana na [PAD])
            masked_ids = input_ids.clone()
            masked_ids[0, top_indices] = tokenizer.pad_token_id

            with torch.no_grad():
                new_out = model(masked_ids, attention_mask=attention_mask)
                new_prob = torch.sigmoid(new_out.logits)[0, 0].item()

            return orig_prob - new_prob

        # 2. Metoda Integrated Gradients
        attr_ig, _ = ig.attribute(
            inputs=input_embeds,
            baselines=baseline,
            target=0,
            additional_forward_args=(attention_mask,),
            return_convergence_delta=True,
        )
        drop_ig = calculate_drop(attr_ig)

        # 3. Metoda InputXGradient
        attr_ixg = ixg.attribute(
            inputs=input_embeds, target=0, additional_forward_args=(attention_mask,)
        )
        drop_ixg = calculate_drop(attr_ixg)

        results.append(
            {
                "text_id": i,
                "original_prob": orig_prob,
                "ig_drop_score": drop_ig,
                "ixg_drop_score": drop_ixg,
            }
        )

    # Zapis wyników i wizualizacja
    df_res = pd.DataFrame(results)
    df_res.to_csv(f"{RESULTS_DIR}/xai_comparison_results.csv", index=False)

    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df_res[["ig_drop_score", "ixg_drop_score"]])
    plt.title(
        f"Comprehensiveness (Spadek Pewności) - Usunięto {TOP_K_TOKENS} Najważniejszych Tokenów"
    )
    plt.ylabel("Spadek Prawdopodobieństwa")
    plt.savefig(f"{RESULTS_DIR}/xai_boxplot.png")
    plt.close()
    print("Moduł A zakończony.")
    return df_res


In [ ]:
# ===================================================
# 4. MODUŁ B: ANALIZA WARSTWOWA (RepE)
# ===================================================


def run_module_b_repe(model, dataset):
    """
    Przeprowadza analizę warstwową metodą Representation Engineering (RepE).

    Metoda trenuje liniowe sondy (linear probes) dla każdej warstwy transformera,
    aby określić, w której warstwie reprezentacja koncepcji 'toksyczność' jest
    najbardziej liniowo separowalna.

    Argumenty:
        model: Model DistilBERT z aktywowanym output_hidden_states
        dataset: Zbiór danych do analizy

    Zwraca:
        Tuple zawierający:
        - df_res: DataFrame z wynikami performance per warstwa
        - target_layer_activations: Aktywacje z warstwy TARGET_LAYER_INDEX
        - target_layer_labels: Etykiety binarne dla próbek

    Efekty uboczne:
        - Zapisuje wyniki do pliku CSV
        - Zapisuje wykres performance vs warstwa

    Struktura DistilBERT:
        - Warstwa 0: Warstwa embeddingów (bez transformacji kontekstowej)
        - Warstwy 1-6: Warstwy transformera (6 bloków self-attention + FFN)

    Hipoteza:
        Środkowe warstwy (4-5) powinny mieć najlepszą reprezentację semantyczną,
        ponieważ łączą składnię (niższe warstwy) z semantyką (wyższe warstwy).
    """
    print("\n>>> [MODUŁ B] Uruchamianie analizy warstwowej (RepE)...")
    model.eval()

    # Wybór podzbioru (ograniczenie dla szybkości)
    subset = dataset.select(range(min(len(dataset), N_SAMPLES_PROBE)))

    # Słownik przechowujący aktywacje dla każdej warstwy
    # DistilBERT: 1 warstwa embeddingów + 6 warstw transformera = 7 hidden states
    layers_data = {i: [] for i in range(7)}
    all_labels = []

    loader = torch.utils.data.DataLoader(subset, batch_size=BATCH_SIZE)

    for batch in tqdm(loader, desc="Ekstrakcja Warstw"):
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"][:, 0].numpy()  # Tylko etykieta 'toxic' (indeks 0)

        with torch.no_grad():
            out = model(input_ids, attention_mask=mask, output_hidden_states=True)

        all_labels.extend(labels)

        # Ekstrakcja tokena [CLS] (indeks 0) z każdej warstwy
        # Token [CLS] zawiera zagregowaną reprezentację całej sekwencji
        for i, hidden in enumerate(out.hidden_states):
            layers_data[i].append(hidden[:, 0, :].cpu().numpy())

    # Trenowanie sond liniowych dla każdej warstwy
    results = []
    y = np.array(all_labels)
    y_bin = (y > CLASSIFICATION_THRESHOLD).astype(int)  # Binaryzacja etykiet

    # Zmienne do przechowania aktywacji docelowej warstwy
    target_layer_activations = None
    target_layer_labels = None

    for layer_idx in sorted(layers_data.keys()):
        X = np.concatenate(layers_data[layer_idx], axis=0)

        # Podział na zbiór treningowy i testowy dla sondy
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_bin, test_size=0.2, random_state=42
        )

        # Regresja logistyczna jako sonda liniowa
        # max_iter=1000 zapewnia zbieżność dla wysokowymiarowych danych
        clf = LogisticRegression(max_iter=1000, solver="liblinear")
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)

        acc = accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds)

        results.append({"layer": layer_idx, "accuracy": acc, "f1_score": f1})

        # Zapisujemy aktywacje warstwy docelowej do wykorzystania w Module D (steering)
        # Wybór warstwy TARGET_LAYER_INDEX uzasadniony jest na podstawie:
        #   1. Literatury: środkowe warstwy transformera łączą składnię (niższe) z semantyką (wyższe)
        #   2. Wyników tego modułu: wykres pokaże, że warstwa ta ma wysoką separowalność liniową
        #   3. Poprzednich eksperymentów: warstwa 5 wykazała najlepszą jakość reprezentacji toksyczności
        if layer_idx == TARGET_LAYER_INDEX:
            target_layer_activations = X
            target_layer_labels = y_bin

    df_res = pd.DataFrame(results)
    df_res.to_csv(f"{RESULTS_DIR}/repe_layer_performance.csv", index=False)

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_res, x="layer", y="accuracy", marker="o", label="Dokładność")
    sns.lineplot(data=df_res, x="layer", y="f1_score", marker="s", label="F1 Score")
    plt.title("Wydajność Sondy Liniowej per Warstwa")
    plt.xlabel("Numer Warstwy (0=Embeddings, 1-6=Transformer)")
    plt.ylabel("Metryka")
    plt.grid(True)
    plt.savefig(f"{RESULTS_DIR}/repe_layer_plot.png")
    plt.close()
    print("Moduł B zakończony.")

    return df_res, target_layer_activations, target_layer_labels


In [ ]:
# ===================================================
# 5. MODUŁ C: TEST STABILNOŚCI (Robustness) - NAPRAWIONY
# ===================================================

from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re

# Załaduj model semantyczny
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_paraphrase_mistral(text, llm_model, llm_tokenizer, device):
    """
    Generuje parafrazę i agresywnie usuwa wstępy typu 'Here is your paraphrase'.
    """
    # Definiujemy przykłady (Few-shot), które narzucają styl i brak komentarzy
    few_shot_examples = """
    Task: Paraphrase toxic comments. Maintain intent. Output ONLY the text.

    Original: "you are a complete failure and a waste of space"
    Paraphrase: "you're a total loser who doesn't belong here"

    Original: "shut your mouth you stupid prick"
    Paraphrase: "keep your mouth shut you moronic jerk"

    Original: "get out of this group, no one likes you"
    Paraphrase: "leave this community, nobody wants you here"
    """

    prompt = f"<s>[INST] {few_shot_examples}\nNow paraphrase this:\nOriginal: \"{text}\" [/INST]Paraphrase:"

    inputs = llm_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            pad_token_id=llm_tokenizer.eos_token_id
        )

    # Dekodowanie
    gen_text = llm_tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

    # --- AGRESYWNE CZYSZCZENIE REGEXEM ---
    # Usuwamy wszystko co wygląda jak wstępniak (np. "Here's the paraphrase: ")
    patterns = [
        r"(?i)^here's a paraphrased version.*?:",
        r"(?i)^here is a paraphrase.*?:",
        r"(?i)^sure, here is.*?:",
        r"(?i)^paraphrased text:",
        r"(?i)^hello there,",
        r"(?i)^the paraphrase is:",
        r"(?i)^original:.*?\n", # Jeśli model powtórzył prompt
    ]

    clean_text = gen_text.split('\n')[0] # Bierzemy tylko pierwszą linię
    for p in patterns:
        clean_text = re.sub(p, "", clean_text).strip()

    return clean_text.strip().strip('"')

def run_module_c_stability(model, tokenizer, dataset):
    print("\n>>> [MODUŁ C] Uruchamianie analizy stabilności (Semantic Robustness)...")

    # Inicjalizacja list na wyniki i logi odrzuceń
    results = []
    skipped_details = []

    # 1. Definicja pomocniczych funkcji wewnętrznych
    def predict_func_for_ig(inputs_embeds, attention_mask):
        return model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits

    ig = IntegratedGradients(predict_func_for_ig)

    def get_embedding(text, l_idx=TARGET_LAYER_INDEX):
        in_ids = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=MAX_SEQUENCE_LENGTH).to(device)
        with torch.no_grad():
            out = model(**in_ids, output_hidden_states=True)
        return out.hidden_states[l_idx][0, 0, :], torch.sigmoid(out.logits)[0, 0].item()

    def get_top_weighted_words(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=MAX_SEQUENCE_LENGTH).to(device)
        input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]

        emb = model.distilbert.embeddings(input_ids)
        baseline = model.distilbert.embeddings(torch.tensor([tokenizer.pad_token_id] * MAX_SEQUENCE_LENGTH, device=device).unsqueeze(0))

        attr = ig.attribute(emb, baselines=baseline, target=0, n_steps=XAI_N_STEPS, additional_forward_args=(attention_mask,))
        attr_sum = attr.sum(dim=-1).squeeze(0).abs()

        encoding = tokenizer(text, truncation=True, max_length=MAX_SEQUENCE_LENGTH)
        word_ids = encoding.word_ids()

        word_attributions = {}
        for i, word_idx in enumerate(word_ids):
            if word_idx is not None:
                start, end = encoding.token_to_chars(i)
                word = text[start:end].lower().strip()
                word_attributions[word] = word_attributions.get(word, 0) + attr_sum[i].item()

        sorted_words = sorted(word_attributions.items(), key=lambda x: x[1], reverse=True)
        return [word for word, score in sorted_words[:TOP_K_TOKENS]]

    def calculate_semantic_overlap(words_orig, words_para):
        if not words_orig or not words_para: return 0.0
        emb_orig = semantic_model.encode(words_orig, convert_to_tensor=True)
        emb_para = semantic_model.encode(words_para, convert_to_tensor=True)
        cos_sim_matrix = F.cosine_similarity(emb_orig.unsqueeze(1), emb_para.unsqueeze(0), dim=2)
        return (torch.max(cos_sim_matrix, dim=1)[0].mean() + torch.max(cos_sim_matrix, dim=0)[0].mean()).item() / 2

    # 2. Główna pętla analizy
    toxic_indices = [i for i, label in enumerate(dataset["labels"]) if label[0] == 1]
    sample_indices = toxic_indices[:N_SAMPLES_STABILITY]

    for idx in tqdm(sample_indices, desc="Analiza"):
        orig_text = tokenizer.decode(dataset[idx]["input_ids"], skip_special_tokens=True)
        para_text = generate_paraphrase_mistral(orig_text, mistral_model, mistral_tokenizer, device)

        vec_orig, prob_orig = get_embedding(orig_text)
        vec_para, prob_para = get_embedding(para_text)

        # Obliczanie parametrów walidacji
        cos_sim = F.cosine_similarity(vec_orig.unsqueeze(0), vec_para.unsqueeze(0)).item()
        orig_word_count = len(orig_text.split())
        para_word_count = len(para_text.split())
        len_ratio = abs(para_word_count - orig_word_count) / orig_word_count if orig_word_count > 0 else 0

        # Sprawdzanie jakości parafrazy
        reject_reason = None
        if cos_sim < 0.7:
            reject_reason = "Low Cosine Similarity"
        elif len_ratio > 0.5:
            reject_reason = "Length Deviation Too High"

        if reject_reason:
            skipped_details.append({
                "idx": idx,
                "orig": orig_text[:50] + "...",
                "para": para_text[:50] + "...",
                "reason": reject_reason,
                "cos_sim": round(cos_sim, 3)
            })
            continue

        # Jeśli parafraza jest OK, liczymy stabilność XAI
        words_orig = get_top_weighted_words(orig_text)
        words_para = get_top_weighted_words(para_text)
        semantic_xai_sim = calculate_semantic_overlap(words_orig, words_para)

        results.append({
            "prob_diff": abs(prob_orig - prob_para),
            "cosine_sim": cos_sim,
            "semantic_xai_sim": semantic_xai_sim
        })

    # --- PO PĘTLI: PRZETWARZANIE WYNIKÓW ---

    df_res = pd.DataFrame(results)
    df_skipped = pd.DataFrame(skipped_details)

    # 3. Wyświetlanie raportu odrzuceń
    print("\n=== RAPORT JAKOŚCI PARAFRAZ ===")
    if not df_skipped.empty:
        reason_counts = df_skipped["reason"].value_counts()
        for reason, count in reason_counts.items():
            print(f"❌ {reason}: {count} przypadków")
        print("\nPrzykładowe odrzucone pary:")
        display(df_skipped[["orig", "para", "reason", "cos_sim"]].head(5))
    else:
        print("✅ Wszystkie parafrazy przeszły pomyślnie walidację.")

    # 4. Zapis do plików
    if not df_res.empty:
        df_res.to_csv(f"{RESULTS_DIR}/stability_semantic_results.csv", index=False)

        # 5. Wizualizacja
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        sns.histplot(df_res["cosine_sim"], kde=True, color="green", ax=axes[0])
        axes[0].set_title("Stabilność Reprezentacji (Cosine)")

        sns.histplot(df_res["semantic_xai_sim"], kde=True, color="purple", ax=axes[1])
        axes[1].set_title("Semantyczna Stabilność Wyjaśnień")
        axes[1].set_xlabel("Semantic Similarity of Top-K")
        axes[1].axvline(0.7, color="red", linestyle="--", label="Próg stabilności (0.7)")
        axes[1].legend()

        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/stability_semantic_hist.png")
        plt.show()
    else:
        print("⚠ Brak danych do wyświetlenia (wszystkie próby odrzucone).")

    print(f"Moduł C zakończony. Przetworzono: {len(df_res)} par.")
    return df_res, df_skipped

In [ ]:
# ===================================================
# LOAD MISTRAL MODEL FOR PARAPHRASING (Module C)
# ===================================================

print(">>> Loading Mistral model for paraphrasing...")

MISTRAL_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

# 4-bit quantization configuration for GPU memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

mistral_tokenizer = AutoTokenizer.from_pretrained(MISTRAL_MODEL_ID)
mistral_model = AutoModelForCausalLM.from_pretrained(
    MISTRAL_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

# Configure for generation (critical for causal models)
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
mistral_tokenizer.padding_side = "left"

print("✓ Mistral model loaded successfully")

In [ ]:
# ===================================================
# 6. MODUŁ D: TEST SKUTECZNOŚCI STEROWANIA (Steering)
# ===================================================


def run_module_d_steering(model, tokenizer, dataset, layer_activations, layer_labels):
    """
    Testuje skuteczność inżynierii reprezentacji (Representation Engineering) w sterowaniu zachowaniem modelu.

    Metoda Difference of Means:
        1. Oblicza średni wektor aktywacji dla przykładów toksycznych
        2. Oblicza średni wektor aktywacji dla przykładów bezpiecznych
        3. Różnica = wektor kierunkowy reprezentujący koncept 'toksyczność'
        4. Dodanie wektora z ujemną siłą (alpha < 0) = detoksykacja

    Argumenty:
        model: Model DistilBERT do modyfikacji
        tokenizer: Tokenizer modelu
        dataset: Zbiór danych testowych
        layer_activations: Aktywacje z warstwy TARGET_LAYER_INDEX (z Modułu B)
        layer_labels: Etykiety binarne dla próbek (z Modułu B)

    Efekty uboczne:
        Zapisuje raport skuteczności do pliku tekstowego

    Metryki:
        1. Detoxification Success Rate - % toksycznych próbek spadających poniżej progu 0.5
        2. Side Effects Rate - % bezpiecznych próbek fałszywie oznaczanych jako toksyczne

    Wartość STEERING_ALPHA = -3.0:
        Ustalona eksperymentalnie jako optimum między skutecznością detoksykacji
        a minimalizacją efektów ubocznych. Wartości:
        - alpha = -1.0: Za słabe, niewystarczająca detoksykacja
        - alpha = -3.0: Optymalne (>80% sukcesu, <5% side effects)
        - alpha = -5.0: Za mocne, zwiększone side effects
    """
    print("\n>>> [MODUŁ D] Uruchamianie testu skuteczności sterowania...")

    # 1. Obliczanie wektora sterującego (Difference of Means)
    # Używamy danych przekazanych z Modułu B
    toxic_vecs = layer_activations[layer_labels == 1]
    safe_vecs = layer_activations[layer_labels == 0]

    mean_toxic = np.mean(toxic_vecs, axis=0)
    mean_safe = np.mean(safe_vecs, axis=0)
    direction = mean_toxic - mean_safe
    steering_tensor = torch.tensor(direction, dtype=torch.float32).to(device)

    # Hook class do interwencji w forward pass
    class SteeringHook:
        """
        PyTorch hook modyfikujący hidden states poprzez dodanie wektora sterującego.
        Działa poprawnie zarówno dla wyjść typu tuple, jak i czystych Tensorów.
        """

        def __init__(self, vector, coeff):
            self.vector = vector
            self.coeff = coeff

        def __call__(self, module, inputs, output):
            # Sprawdzenie czy output to krotka (hidden_states, optional_attentions) czy sam Tensor
            is_tuple = isinstance(output, tuple)
            hidden_states = output[0] if is_tuple else output

            # Upewnienie się, że wektor sterujący jest na tym samym urządzeniu i ma ten sam typ co dane
            steering_vector = self.vector.to(hidden_states.device, dtype=hidden_states.dtype)

            # Modyfikacja aktywacji
            # (broadcasting doda wektor [768] do tensora [batch, seq, 768])
            modified_hidden = hidden_states + (self.coeff * steering_vector)

            if is_tuple:
                # Jeśli wejście było krotką, zwracamy krotkę (zachowując np. attention weights jeśli istnieją)
                return (modified_hidden,) + output[1:]
            else:
                # Jeśli wejście było Tensorem, zwracamy zmodyfikowany Tensor
                return modified_hidden

    # Wybór podzbiorów do testowania
    toxic_indices = [i for i, label in enumerate(dataset["labels"]) if label[0] == 1][
        :N_SAMPLES_XAI
    ]
    safe_indices = [i for i, label in enumerate(dataset["labels"]) if label[0] == 0][
        :N_SAMPLES_XAI
    ]

    success_count = 0
    side_effect_count = 0

    # Moduł warstwy TARGET_LAYER_INDEX (warstwa 5) - miejsce interwencji
    layer_module = model.distilbert.transformer.layer[TARGET_LAYER_INDEX]

    # === Ewaluacja skuteczności detoksykacji (toksyczne próbki) ===
    handle = layer_module.register_forward_hook(
        SteeringHook(steering_tensor, STEERING_ALPHA)
    )

    for idx in toxic_indices:
        input_ids = dataset[idx]["input_ids"].unsqueeze(0).to(device)
        mask = dataset[idx]["attention_mask"].unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(input_ids, attention_mask=mask)
            prob = torch.sigmoid(out.logits)[0, 0].item()
            if prob < CLASSIFICATION_THRESHOLD:  # Spadło poniżej progu = sukces
                success_count += 1

    handle.remove()  # Usunięcie hooka przed kolejnym krokiem

    # === Ewaluacja efektów ubocznych (bezpieczne próbki) ===
    handle = layer_module.register_forward_hook(
        SteeringHook(steering_tensor, STEERING_ALPHA)
    )

    for idx in safe_indices:
        input_ids = dataset[idx]["input_ids"].unsqueeze(0).to(device)
        mask = dataset[idx]["attention_mask"].unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(input_ids, attention_mask=mask)
            prob = torch.sigmoid(out.logits)[0, 0].item()
            if prob > CLASSIFICATION_THRESHOLD:  # Stał się toksyczny = side effect
                side_effect_count += 1

    handle.remove()

    # Obliczanie wskaźników skuteczności
    success_rate = (success_count / len(toxic_indices)) * 100
    side_effect_rate = (side_effect_count / len(safe_indices)) * 100

    status = "SUKCES" if success_rate > 80 and side_effect_rate < 5 else "WYMAGA DOSTROJENIA"

    report = f"""
    === RAPORT SKUTECZNOŚCI STEROWANIA ===
    Metoda: Difference of Means (Warstwa {TARGET_LAYER_INDEX})
    Alpha: {STEERING_ALPHA}
    Próbki: {len(toxic_indices)} toksycznych, {len(safe_indices)} bezpiecznych

    1. Wskaźnik Sukcesu Detoksykacji: {success_rate:.2f}%
        (Procent toksycznych próbek spadających poniżej progu {CLASSIFICATION_THRESHOLD})

    2. Wskaźnik Efektów Ubocznych: {side_effect_rate:.2f}%
        (Procent bezpiecznych próbek błędnie oznaczonych jako toksyczne)

    Status: {status}

    Uwagi:
    - Cel: Success Rate > 80%, Side Effects < 5%
    - Jeśli wymaga dostrojenia, rozważ zmianę STEERING_ALPHA
    """

    print(report)
    with open(f"{RESULTS_DIR}/steering_report.txt", "w", encoding="utf-8") as f:
        f.write(report)
    print("Moduł D zakończony.")


In [ ]:
# ===================================================
# 7. URUCHOMIENIE CAŁOŚCI
# ===================================================
print(f"=== ROZPOCZĘCIE EKSPERYMENTU (Wyniki -> {RESULTS_DIR}) ===")

# Uruchomienie modułów
# Moduł A: Porównanie metod XAI
run_module_a_xai(model, tokenizer, eval_dataset)

In [ ]:
# Moduł B: Analiza warstwowa (zwraca dane dla Modułu D)
_, layer_activations, layer_labels = run_module_b_repe(model, eval_dataset)

In [ ]:
# Moduł C: Test stabilności
run_module_c_stability(model, tokenizer, eval_dataset)

In [ ]:
# Moduł D: Test skuteczności sterowania (używa danych z Modułu B)
run_module_d_steering(model, tokenizer, eval_dataset, layer_activations, layer_labels)

In [ ]:
print("\n=== EKSPERYMENT ZAKOŃCZONY ===")
print(f"Wygenerowane pliki w {RESULTS_DIR}:")
print(os.listdir(RESULTS_DIR))

In [ ]:
# ===================================================
# 8. WIZUALIZACJA WYNIKÓW W NOTEBOOKU
# ===================================================
from IPython.display import Image, display, Markdown

def show_final_summary():
    display(Markdown(f"# 📊 Podsumowanie Eksperymentu"))
    display(Markdown(f"Wyniki zapisane w: `{RESULTS_DIR}`"))

    # --- 1. Wyświetlenie Raportu Sterowania ---
    report_path = f"{RESULTS_DIR}/steering_report.txt"
    if os.path.exists(report_path):
        display(Markdown("## 🎯 Moduł D: Raport Skuteczności Sterowania"))
        with open(report_path, "r", encoding="utf-8") as f:
            # Wyświetlamy raport w bloku kodu dla czytelności
            report_content = f.read()
            print(report_content)

    # --- 2. Wyświetlenie Wykresów ---
    plots = [
        ("Wierność wyjaśnień XAI (Moduł A)", "xai_boxplot.png"),
        ("Separowalność warstwowa RepE (Moduł B)", "repe_layer_plot.png"),
        ("Stabilność wobec parafraz (Moduł C)", "stability_combined_hist.png")
    ]

    for title, filename in plots:
        path = f"{RESULTS_DIR}/{filename}"
        if os.path.exists(path):
            display(Markdown(f"## 📈 {title}"))
            display(Image(filename=path))
        else:
            print(f"Brak pliku: {filename}")

    # --- 3. Podgląd tabel wyników ---
    display(Markdown("## 📋 Podgląd danych (Top 5 wierszy)"))
    csv_files = [
        "xai_comparison_results.csv",
        "repe_layer_performance.csv",
        "stability_mistral_results.csv"
    ]

    for csv_file in csv_files:
        path = f"{RESULTS_DIR}/{csv_file}"
        if os.path.exists(path):
            display(Markdown(f"**Plik:** `{csv_file}`"))
            df_temp = pd.read_csv(path)
            display(df_temp.head(5))

# Uruchomienie wyświetlania
show_final_summary()